In [12]:
from openai import OpenAI
import pandas as pd
import requests
from datetime import datetime
from pprint import pprint
import tiktoken
from pypdf import PdfReader
from IPython.display import Image, Markdown, display
import os
import matplotlib.image as mpimg
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [16]:
pd.set_option('display.max_colwidth', None)

def pp(df):
    return display( df.style.set_properties(subset=['emails'], **{'text-align': 'left', 'white-space': 'pre-wrap', 'width': '900px'}) )

In [17]:
columns = ['reviews', 'emails']
df = pd.DataFrame(columns=columns)
df['reviews'] = [
    "Nice socks, great colors, just enough support for wearing with a good pair of sneakers.",
    "Love Deborah Harness's Trilogy! Didn't want the story to end and hope they turn this trilogy into a movie. I would love it if she wrote more books to continue this story!!!",
    "SO much quieter than other compressors. VERY quick as well. You will not regret this purchase.",
    "Shirt a bit too long, with heavy hem, which inhibits turning over. I cut off the bottom two inches all around, and am now somewhat comfortable. Overall, material is a bit too heavy for my liking.",
    "The quality on these speakers is insanely good and doesn't sound muddy when adjusting bass. Very happy with these.",
    "Beautiful watch face. The band looks nice all around. The links do make that squeaky cheapo noise when you swing it back and forth on your wrist which can be embarrassing in front of watch enthusiasts. However, to the naked eye from afar, you can't tell the links are cheap or folded because it is well polished and brushed and the folds are pretty tight for the most part. love the new member of my collection and it looks great. I've had it for about a week and so far it has kept good time despite day 1 which is typical of a new mechanical watch."
]
df.head()

,reviews,emails
0,"Nice socks, great colors, just enough support for wearing with a good pair of sneakers.",NaN
1,Love Deborah Harness's Trilogy! Didn't want the story to end and hope they turn this trilogy into a movie. I would love it if she wrote more books to continue this story!!!,NaN
2,SO much quieter than other compressors. VERY quick as well. You will not regret this purchase.,NaN
3,"Shirt a bit too long, with heavy hem, which inhibits turning over. I cut off the bottom two inches all around, and am now somewhat comfortable. Overall, material is a bit too heavy for my liking.",NaN
4,The quality on these speakers is insanely good and doesn't sound muddy when adjusting bass. Very happy with these.,NaN


In [19]:
chat = [{"role": "system", "content": "You are a polite customer support representative."}]
postfix = "\n\nWrite an email to customers to address the issues put forward in the above review, thank them if they write good comments, and encourage them to make further purchases. Do not give promotion codes or discounts to the customers. Do not recommend other products. Keep the emails short."
def getMail(review):
  chat_history = chat.copy()
  chat_history.append({"role":"user", "content":review+postfix})

  reply = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=chat_history
    )

  return reply.choices[0].message.content

df['emails'] = df.apply(lambda x: getMail(x.reviews), axis=1)
pp(df)

,reviews,emails
0,"Nice socks, great colors, just enough support for wearing with a good pair of sneakers.","Dear [Customer's Name], Thank you for taking the time to leave a wonderful review about our socks! We appreciate your kind words and are glad to hear that you are satisfied with the colors and support they provide, especially when wearing them with your sneakers. At [Company Name], customer feedback is highly valued, and it motivates us to keep providing excellent products and service. We genuinely appreciate your support and encourage you to share your positive experience with friends and family who might also benefit from our socks. Should you have any further questions or need assistance with any future purchases, please don't hesitate to reach out. We're here to help! Once again, thank you for choosing our socks, and we look forward to serving you in the future. Warm regards, [Your Name] [Customer Support Representative] [Company Name]"
1,Love Deborah Harness's Trilogy! Didn't want the story to end and hope they turn this trilogy into a movie. I would love it if she wrote more books to continue this story!!!,"Dear [Customer Name], Thank you so much for taking the time to share your wonderful feedback about Deborah Harness's trilogy! We truly appreciate your support and are thrilled to hear that you enjoyed the story so much. It's always gratifying to know that our books have captivated our readers. We understand your desire for the story to continue, and we'll be sure to pass along your suggestion to the author. While we cannot make any promises about future books or adaptations, we're grateful for your enthusiasm and we'll continue to bring you captivating stories from talented authors. Once again, thank you for your kind words and for being a valued customer. If you have any other questions or need assistance with anything else, please don't hesitate to reach out. Warm regards, [Your Name] Customer Support Representative"
2,SO much quieter than other compressors. VERY quick as well. You will not regret this purchase.,"Dear valued customer, Thank you for taking the time to share your experience with us regarding the noise level and speed of our compressor. We truly appreciate your positive feedback and are delighted to know that you are satisfied with your purchase. At our company, we strive to provide products that exceed our customers' expectations in both functionality and performance. We understand the importance of a quiet and efficient compressor, and we are glad that our product met your requirements. We want to express our gratitude for your support and trust in our brand. Your kind words mean a lot to us, and it encourages us to continue delivering high-quality products that bring satisfaction to our customers. If you ever need any further assistance or have any questions in the future, please do not hesitate to reach out to us. We are here to help. Thank you again for choosing our compressor, and we look forward to serving you again in the future. Best regards, [Your Name] Customer Support Representative"
3,"Shirt a bit too long, with heavy hem, which inhibits turning over. I cut off the bottom two inches all around, and am now somewhat comfortable. Overall, material is a bit too heavy for my liking.","Dear Valued Customer, Thank you for taking the time to provide us with your honest feedback on our shirt. We truly appreciate it. We apologize for any inconvenience caused by the length of the shirt and the heavy hem. We understand that this can hinder the ability to turn it over comfortably. Your suggestion of cutting off the bottom two inches all around is a clever solution, and we're glad to hear that it has improved your comfort. We also take note of your preference for a lighter material. We understand that personal comfort is key when it comes to clothing, and we appreciate your input on this matter. At [Company Name], we constantly strive to improve our products and make them more suitable for our cu